# 102 Category Flower Dataset 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"GPU/CPU: {device}")

GPU/CPU: cpu


In [2]:
batch_size = 64
lr = 0.001
num_epochs = 20
num_classes = 102

## 1. 전처리(이미지)

In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()    
])

## 2. 데이터 불러오기

In [5]:
train_dataset = datasets.Flowers102(root="data", split="train", transform=train_transforms, download=True)
val_dataset = datasets.Flowers102(root="data", split="val", transform=val_transforms, download=True)
test_dataset = datasets.Flowers102(root="data", split="test", transform=val_transforms, download=True)

100.0%
100.0%
100.0%


In [6]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
print(f"훈련 데이터: {len(train_loader)}")
print(f"검증 데이터: {len(val_loader)}")
print(f"테스트 데이터: {len(test_loader)}")

훈련 데이터: 16
검증 데이터: 16
테스트 데이터: 97


## 모델 설계(4계층) 학습

In [8]:
# 변경하고 코드 전체 수정해보세요.
class Flower102CNN(nn.Module):
    def __init__(self):
        super(Flower102CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(64 * 6 * 6, 512)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x

model = Flower102CNN().to(device)

In [9]:
# 3. 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 학습 루프
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # 순전파
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# 5. 학습 및 평가
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x3136 and 2304x512)

## 평가

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
evaluate_model(model, test_loader)